In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv

# Path to the ChromeDriver executable
chromedriver_path = 'C:/Users/Amal Maatoug/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe' 


service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)


url = 'https://fr.shopping.rakuten.com/nav/Tel-PDA_Telephones-mobiles'

# Ouvrir l'URL
driver.get(url)

        
time.sleep(20)

# Initialiser les données
data = []

# Fonction pour extraire les données des produits de la page actuelle
def extract_data():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    selector = '.mh-16.productList_childrenLayout_d73'
    containers = soup.select(selector)
    
    for product in containers:
        title_tag = product.find('p', class_='description_styleTitle_KPO lh-title normal mv0 b di')  
        neuf_price_tag = product.find('span', class_='f20 dib b')
        occasion_price_tag = product.find('span', class_='b dib f20')
        description_tag=product.find('ul',class_='description_bulletPoints_qgD')

        title = title_tag.text.strip() if title_tag else 'N/A'
        neuf_price = neuf_price_tag.text.strip() if neuf_price_tag else 'N/A'
        occasion_price = occasion_price_tag.text.strip() if occasion_price_tag else 'N/A'
        description=description_tag.text.strip() if description_tag else 'N/A'

        data.append({
            'title': title,
            'neuf_price': neuf_price,
            'occasion_price': occasion_price,
            'description':description
        })


# Extraire les données de la première page
extract_data()

# Fermer le navigateur
driver.quit()

# Définir le chemin du fichier CSV
csv_file_path = 'extracted_data.csv'

# Écrire les données dans un fichier CSV
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['title', 'neuf_price', 'occasion_price','description'])
    writer.writeheader()
    writer.writerows(data)

print(f"Les données ont été écrites dans {csv_file_path}")


Les données ont été écrites dans extracted_data.csv


In [24]:

# Définir le chemin du fichier CSV
csv_file_path = 'extracted_data.csv'

# Écrire les données dans un fichier CSV
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['title', 'neuf_price', 'occasion_price','description'])
    writer.writeheader()
    writer.writerows(data)

print(f"Les données ont été écrites dans {csv_file_path}")

Les données ont été écrites dans extracted_data.csv
